# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps
import requests 
import os
import json
import time

# Import API key

from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
weather_df = pd.read_csv("city_data.csv")
weather_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kushiro,JP,2021-07-27 04:04:10,42.9750,144.3747,291.92,97,100,4.76
1,1,Bulawayo,ZW,2021-07-27 04:02:46,-20.1500,28.5833,278.67,74,0,4.47
2,2,Kodiak,US,2021-07-27 04:03:17,57.7900,-152.4072,287.43,76,44,2.31
3,3,Hobart,AU,2021-07-27 04:01:58,-42.8794,147.3294,290.07,54,75,1.79
4,4,New Norfolk,AU,2021-07-27 04:04:13,-42.7826,147.0587,289.87,55,69,2.68
5,5,Lensk,RU,2021-07-27 04:04:14,60.7253,114.9278,305.56,28,4,2.93
6,6,Bethel,US,2021-07-27 04:03:18,41.3712,-73.4140,296.28,87,1,1.48
7,7,Pechenga,RU,2021-07-27 04:04:15,69.5259,31.1703,282.63,78,20,1.46
8,8,Rikitea,PF,2021-07-27 04:03:07,-23.1203,-134.9692,296.99,79,99,8.33
9,9,Fatwa,IN,2021-07-27 04:04:17,25.5167,85.3167,299.20,63,100,1.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(gkey)

In [12]:
location = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"]

In [22]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2.1)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
refined_weather_df = pd.DataFrame(weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (refined_weather_df["Max Temp"] <= 80) & (refined_weather_df["Max Temp"] > 70)
wind_speed = (refined_weather_df["Wind Speed"] < 10)
cloudiness = (refined_weather_df["Cloudiness"] == 0)

refined_weather_df

,City,Max Temp,Wind Speed,Cloudiness
0,Kushiro,291.92,4.76,100
1,Bulawayo,278.67,4.47,0
2,Kodiak,287.43,2.31,44
3,Hobart,290.07,1.79,75
4,New Norfolk,289.87,2.68,69
...,...,...,...,...
548,Rawson,277.64,12.75,89
549,Hede,298.92,11.00,100
550,Puerto Baquerizo Moreno,294.59,4.89,59
551,Kodinsk,303.64,3.30,2


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotel_df = pd.DataFrame(weather_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Kushiro,JP,42.9750,144.3747,
1,Bulawayo,ZW,-20.1500,28.5833,
2,Kodiak,US,57.7900,-152.4072,
3,Hobart,AU,-42.8794,147.3294,
4,New Norfolk,AU,-42.7826,147.0587,
...,...,...,...,...,...
548,Rawson,AR,-43.3002,-65.1023,
549,Hede,CN,33.7681,120.2517,
550,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,
551,Kodinsk,RU,58.6881,99.1844,


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

# Display figure

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))